In [5]:
import numpy as np
import math

## Extract Features

In [9]:
!python -m tefla.predict --model ../tefla/examples/imagenet_tl_feature_extract/vgg16.py \
--training_cnf ../tefla/examples/imagenet_tl_feature_extract/train_cnf.py \
--predict_dir ../../data/clothing/koovs_tefla_ready/test/ --tag test_new --weights_from vgg/vgg_16.ckpt \
--output_layer vgg_16/conv5/maxpool5 --predict_type 1_crop

Traceback (most recent call last):
  File "/usr/lib64/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/tefla/predict.py", line 6, in <module>
    from tefla.core.iter_ops import create_prediction_iter, convert_preprocessor
ImportError: No module named core.iter_ops


In [ ]:
!python -m tefla.predict_withf --model examples/imagenet_tl_feature_extract/bottleneck_model.py \
--training_cnf examples/imagenet_tl_feature_extract/train_cnf.py \
--images_dir ../../data/clothing/koovs_tefla_ready/test/ --tag test_new --weights_from weights/model-epoch-9.ckpt \
--features_file ../../data/clothing/koovs_tefla_ready/predictions/test_new/features.npy   

## Similarity Algorithm

In [6]:
classes = ['train', 'val']
target = 'test'
weights = range(24)
with open('/home/ec2-user/data/clothing/koovs_tefla_ready/predictions/' + target + '/predictions/' + target +  '/predictions.csv','r') as f:
    count = 0
    for line in f:
        if count==4:
            res = line
        count += 1
res_arr = res.split(",")
result_class = np.argmax(res_arr[1:])
target_index = [i[0] for i in sorted(enumerate(res_arr[1:]), key=lambda x:x[1])]
min_err = 100
for class_ in classes:
    count = 0
    with open('/home/ec2-user/data/clothing/koovs_tefla_ready/predictions/' + class_ + '/predictions.csv','r') as f:
        for line in f:
            error = 0
            if count >0 and np.argmax(line.split(",")[1:]) == result_class:
                line_index = [i[0] for i in sorted(enumerate(line.split(",")[1:]), key=lambda x:x[1])]
                
                for i in range(23):
                    error += float(line.split(",")[1:][line_index[i]])*weights[i]-float(res_arr[1:][line_index[i]])*weights[i]
                if math.fabs(error) <= math.fabs(min_err):
                    min_err = error
                    print line.split(",")[0],res_arr[0],min_err,class_
            count += 1
            
              

img_1010 img_1033 -3.89358139272 train
img_1012 img_1033 0.0660434126741 train
